# 📚 2.1 Genre Prediction using BERT
This notebook fine-tunes a BERT model to predict genres from book title and description.

In [1]:
# ✅ Install required packages (if running outside Docker, else skip)
!pip install transformers scikit-learn torch pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 9.0 MB/s eta 0:00:00:00:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 9.2 MB/s eta 0:00:00:00:010:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 9.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 8.4 MB/s eta 0:00:0000:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 9.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 9.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 🔹 Step 1: Load and Preprocess Data

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

# Load your cleaned dataset
df = pd.read_csv("archive/cleaned/for_training_no_nulls.csv")

# Convert genre string to list
df['genre_list'] = df['genre'].apply(lambda x: [g.strip() for g in x.split(',')])
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['genre_list'])

# Prepare input text
df['text'] = df['title'] + ' ' + df['description']
X = df['text'].tolist()

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


## 🔹 Step 2: Tokenize Text

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    }

train_data = tokenize_data(X_train, y_train)
val_data = tokenize_data(X_val, y_val)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## 🔹 Step 3: Create Dataset Class

In [5]:
import torch
from torch.utils.data import Dataset

class BookGenreDataset(Dataset):
    def __init__(self, data):
        self.input_ids = torch.tensor(data['input_ids'])
        self.attention_mask = torch.tensor(data['attention_mask'])
        self.labels = torch.tensor(data['labels'], dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

train_dataset = BookGenreDataset(train_data)
val_dataset = BookGenreDataset(val_data)


## 🔹 Step 4: Fine-tune BERT

In [7]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=y.shape[1],
    problem_type='multi_label_classification'
)

training_args = TrainingArguments(
    output_dir='./results',
    # evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

## 🔹 Step 5: Train the Model

In [ ]:
trainer.train()

## 🔹 Step 6: Save Model

In [ ]:
model.save_pretrained("book_genre_model")
tokenizer.save_pretrained("book_genre_model")


In [11]:
# Fix: Ensure pathlib.Path is imported
from pathlib import Path

# Update the requirements.txt to explicitly include the latest transformers version
req_path = Path("requirements.txt")
requirements = req_path.read_text().splitlines()

# Update or append transformers>=4.41.2
new_reqs = []
transformers_updated = False
for line in requirements:
    if line.strip().startswith("transformers"):
        new_reqs.append("transformers>=4.41.2")
        transformers_updated = True
    else:
        new_reqs.append(line)

if not transformers_updated:
    new_reqs.append("transformers>=4.41.2")

# Write updated requirements.txt
req_path.write_text("\n".join(new_reqs) + "\n")
req_path.name


'requirements.txt'